In [1]:
import numpy as np
from que_onda import *
import tensorflow as tf

np.set_printoptions(floatmode='unique')
dtype = tf.float32
rng = np.random.default_rng()

2025-03-26 19:12:55.736548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
song = 'MusicDelta_80sRock'
# song = 'MusicDelta_Rock'

channels, samplerate = io.read_channels(f'data/MedleyDB_V1/{song}_MIX.mp3')


In [3]:
name = 'sym6'
n_levels = 10
make = wavelets.sym6
wavelet = make(dtype)
filters = wavelets.dwt_filters(wavelet)
inverse_filters = wavelets.idwt_filters(wavelet)

levels = wavelets.wavedec_valid(filters, n_levels, channels)
print(f'Levels: {[tuple(l.shape) for l in levels]}')
frames = framing.levels_to_frames(levels)
print(f'Frames: {[tuple(l.shape) for l in frames]}')
frame_level_rms = analysis.rms_by_frame_and_level(frames)
print(f'Frame/Level RMS: {tuple(frame_level_rms.shape)}')

distances = analysis.distance_matrix(frame_level_rms)


Levels: [(2, 3170), (2, 3170), (2, 6340), (2, 12680), (2, 25360), (2, 50720), (2, 101440), (2, 202880), (2, 405760), (2, 811520)]
Frames: [(3170, 2, 1), (3170, 2, 1), (3170, 2, 2), (3170, 2, 4), (3170, 2, 8), (3170, 2, 16), (3170, 2, 32), (3170, 2, 64), (3170, 2, 128), (3170, 2, 256)]
Frame/Level RMS: (3170, 10)


2025-03-26 19:13:15.849156: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40195600 exceeds 10% of free system memory.
2025-03-26 19:13:15.941945: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40195600 exceeds 10% of free system memory.
2025-03-26 19:13:16.013989: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40195600 exceeds 10% of free system memory.
2025-03-26 19:13:16.059812: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40195600 exceeds 10% of free system memory.
2025-03-26 19:13:16.123091: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40195600 exceeds 10% of free system memory.


In [11]:
int_distances = defrag.integer_distance_matrix(distances)
unique_distances = tf.unique(tf.reshape(distances, [-1]))[0].shape[0]
print(f'unique distances: {unique_distances}')
unique_int_distances = np.unique(int_distances).shape[0]
print(f'unique integer distances: {unique_int_distances}')
loss = unique_distances - unique_int_distances
print(f'loss: {loss} ({loss * 100 / unique_distances}%)')


unique distances: 4903862
unique integer distances: 4847889
loss: 55973 (1.141406507768775%)


In [12]:
route = defrag.defrag(distances)


I0000 00:00:1743016951.493146     273 search.cc:285] Start search (memory used = 720.80 MB)
I0000 00:00:1743016951.503441     273 search.cc:285] Root node processed (time = 8 ms, constraints = 12688, memory used = 720.80 MB)
I0000 00:00:1743016956.898605     273 search.cc:285] Solution #0 (144759701707, time = 5403 ms, branches = 34, failures = 1, depth = 33, memory used = 720.80 MB, limit = 2%)
I0000 00:00:1743016965.127715     273 search.cc:285] Solution #1 (144755766875, maximum = 144759701707, time = 13632 ms, branches = 38, failures = 3, depth = 33, LinKernighan, neighbors = 1, filtered neighbors = 1, accepted neighbors = 1, memory used = 720.80 MB, limit = 2%)
I0000 00:00:1743016965.148274     273 search.cc:285] Solution #2 (144742220093, maximum = 144759701707, time = 13653 ms, branches = 43, failures = 5, depth = 33, LinKernighan, neighbors = 2, filtered neighbors = 2, accepted neighbors = 2, memory used = 720.80 MB, limit = 2%)
I0000 00:00:1743016965.167748     273 search.cc:2

In [13]:
defragged = framing.remix(frames, route)
df_levels = framing.frames_to_levels(defragged)
df_channels = wavelets.waverec(inverse_filters, df_levels)
io.write_channels(f'temp/{song}-{name}-{n_levels}.mp3', df_channels, samplerate)
